In [211]:
lat = 40.229730557967
lon = -74.002934930983

profile = [
    {
        "key": "natural",
        "value": "beach",
        "distance_within": 15,
        "type": "bicycle",
        "weight": 20
    },
    {
        "key": "name",
        "value": "Newark Penn Station",
        "distance_within": 60,
        "type": "auto",
        "weight": 10
    },
    {
        "key": "name",
        "value": "Hurley School (historical)",
        "distance_within": 20,
        "type": "auto",
        "weight": 10
    },
    {
        "key": "name",
        "value": "Sandy Hook Lighthouse",
        "distance_within": 30,
        "type": "auto",
        "weight": 10
    },
    {
        "key": "amenity",
        "value": "pub",
        "distance_within": 5,
        "type": "pedestrian",
        "weight": 20
    },
    {
        "key": "amenity",
        "value": "cafe",
        "distance_within": 5,
        "type": "pedestrian",
        "weight": 20
    },
    {
        "key": "amenity",
        "value": "restaurant",
        "distance_within": 5,
        "type": "pedestrian",
        "weight": 20
    },
    {
        "key": "highway",
        "value": "cycleway",
        "distance_within": 10,
        "type": "bicycle",
        "weight": 20
    },
]

In [212]:
query_fields = """
[
'access',
'addr:housename',
'addr:housenumber',
'addr:interpolation',
'admin_level',
'aerialway',
'aeroway',
'amenity',
'area',
'barrier',
'bicycle',
'brand',
'bridge',
'boundary',
'building',
'construction',
'covered',
'culvert',
'cutting',
'denomination',
'disused',
'embankment',
'foot',
'generator:source',
'harbour',
'highway',
'historic',
'horse',
'intermittent',
'junction',
'landuse',
'layer',
'leisure',
'lock',
'man_made',
'military',
'motorcar',
'name',
'natural',
'office',
'oneway',
'operator',
'place',
'population',
'power',
'power_source',
'public_transport',
'railway',
'ref',
'religion',
'route',
'service',
'shop',
'sport',
'surface',
'toll',
'tourism',
'tower:type',
'tunnel',
'water',
'waterway',
'wetland',
'width',
'wood'
]
"""


In [213]:
import psycopg2
def get_buffer(lat, lon, key_query, distance_in_meters):

    conn = psycopg2.connect(host="postgres",database="osm", user="osm", password="osm")
    cursor = conn.cursor()
    query_fields_2 = query_fields.replace('\'', '"')
    
    cursor.execute("""
      SELECT name, keys,
        ST_X(closest_point) as lon,
        ST_Y(closest_point) as lat,
        distance
      FROM (
        SELECT *, ST_Transform(ST_ClosestPoint(way, 
        ST_TRANSFORM(
                 ST_SETSRID(
                     ST_GeomFromText(
                         'POINT(%f %f)'
                     ),
                 4326),
             3857)), 4326) as closest_point,
        ST_DISTANCE(
            way,
             ST_TRANSFORM(
                 ST_SETSRID(
                     ST_GeomFromText(
                         'POINT(%f %f)'
                     ),
                 4326),
             3857)
         ) AS distance
        FROM (
        SELECT osm_id, name, way,
            hstore(ARRAY%s, ARRAY%s) as keys
        from planet_osm_polygon
        UNION ALL
        SELECT osm_id, name, way,
            hstore(ARRAY%s, ARRAY%s) as keys
        from planet_osm_point
        UNION ALL
        SELECT osm_id, name, way,
            hstore(ARRAY%s, ARRAY%s) as keys
        from planet_osm_line) osm WHERE
        ST_DWithin(way,
             ST_TRANSFORM(
                 ST_SETSRID(
                     ST_GeomFromText(
                         'POINT(%f %f)'
                     ),
                 4326),
             3857), %f)
        AND %s
        ) nearby ORDER BY DISTANCE LIMIT 2
    """ % (lon, lat, lon, lat,
           query_fields, query_fields_2, 
           query_fields, query_fields_2, 
           query_fields, query_fields_2, 
           lon, lat, distance_in_meters,
           key_query))
    #select * FROM (
    #    select
    #     *,
    #     ST_X(ST_Transform(st_centroid(way), 4326)) as lon,
    #     ST_Y(ST_Transform(st_centroid(way), 4326)) as lat,
    #     ST_DISTANCE(
    #         way,
    #         ST_TRANSFORM(
    #             ST_SETSRID(
    #                 ST_GeomFromText(
    #                     'POINT(%f %f)'
    #                 ),
    #             4326),
    #         3857)
    #     ) AS distance
    #    FROM
    #       planet_osm_polygon
    #) AS subquery WHERE distance < %f 
    #ORDER BY distance;
    #""" % (lon, lat, distance_in_meters))
    returned_data = []
    for record in cursor:
        returned_data.append(record)

    cursor.close()
    return returned_data

costing_multiplier = {
    'auto': 100,
    'bicycle': 30,
    'pedestrian': 10
}

scores = list()
for setting in profile:
    key_query = "keys->'%s' = '%s'" % (setting['key'], setting['value'])
    distance_in_meters = setting['distance_within'] * costing_multiplier[setting['type']] * 100
    nearby = get_buffer(lat, lon, key_query, distance_in_meters)
    if len(nearby) == 0:
        nearby = [[None, None, None, None, None]]
    print (key_query, distance_in_meters, nearby[0][4])
    scores.append({'profile': setting, 'name': nearby[0][0],'lon': nearby[0][2], 'lat': nearby[0][3]})
    
#print (scores)

keys->'natural' = 'beach' 45000 663.919528442108
keys->'name' = 'Newark Penn Station' 600000 75817.8832295781
keys->'name' = 'Hurley School (historical)' 200000 14263.8246985781
keys->'name' = 'Sandy Hook Lighthouse' 300000 33883.9816776153
keys->'amenity' = 'pub' 5000 651.249237072539
keys->'amenity' = 'cafe' 5000 827.184895398681
keys->'amenity' = 'restaurant' 5000 710.12369094896
keys->'highway' = 'cycleway' 30000 6240.16576978148


In [214]:
# Load the path from Valhalla

import requests
import json

def get_routed_distance(o_lon, o_lat, d_lon, d_lat, costing):
    url = "http://valhalla:8002/route"
    data =  {
        "locations":[
            {
                "lat":o_lat,
                "lon":o_lon,
                "type":"break"
            },{
                "lat":d_lat,
                "lon":d_lon,
                "type":"break"
            }
        ],
        "costing":costing,
        "directions_options":{
            "units":"miles"
        }
    }

    data_json = json.dumps(data)
    response = requests.post(url, data=data_json)
    response_obj = json.loads(response.text)
    #distance = response_obj['trip']['summary']['length']
    time_in_seconds = response_obj['trip']['summary']['time']
    #west = response_obj['trip']['summary']['min_lon']
    #south = response_obj['trip']['summary']['min_lat']
    #east = response_obj['trip']['summary']['max_lon']
    #north = response_obj['trip']['summary']['max_lat']
    #geometry = decode_geometry(response_obj['trip']['legs'][0]['shape'])
    #print (distance)
    #print (time_in_seconds)
    return time_in_seconds

In [215]:
total_weights = 0
total_weighted_values = 0
for score in scores:
    ideal_distance = (score['profile']['distance_within'] * 60)
    value = 0
    if (score['lon'] and score['lat']):
        time = get_routed_distance(lon, lat, score['lon'], score['lat'], score['profile']['type'])

        if time > ideal_distance:
            value = 100 - (((time - ideal_distance) / 60) * 2)
        elif time < ideal_distance:
            value = 100 + (((ideal_distance - time) / 60) * 0.5)
        else:
            value = 100

        if value > 200:
            value = 200
        elif value < 0:
            value = 0

    score['time'] = time
    score['ideal_time'] = ideal_distance
    score['value'] = value
    score['weighted_time'] = value * score['profile']['weight']
    total_weights = total_weights + score['profile']['weight']
    total_weighted_values = total_weighted_values + score['weighted_time']
    print(score['profile']['key'],score['profile']['value'],score['profile']['weight'],score['value'])
        
            
final_score = (total_weighted_values / total_weights)
print('-------------------------')                                                          
print (final_score)

natural beach 20 106.48333333333333
name Newark Penn Station 10 102.25833333333334
name Hurley School (historical) 10 103.35
name Sandy Hook Lighthouse 10 89.16666666666667
amenity pub 20 91.06666666666666
amenity cafe 20 92.06666666666666
amenity restaurant 20 88.6
highway cycleway 20 68.7
-------------------------
91.43141025641026
